# Checkpointing in Workflows

During the execution of a Workflow, `Checkpoint` objects are automatically stored within the `Context` class. By default, `Checkpoints` are automatically created at the completion of a step. At such instants in the Workflow execution, we checkpoint the name of the last completed step, it's input `Event`, it's output `Event`, as well as a snapshot of the `Context` itself. We also store a `Checkpoint` before the emission of the `StartEvent`, but note that running from these special checkpoints are essentially equivalent to the typical execution of a `Workflow` -- we add it here purely for convenience.

In this notebook, we demonstrate how to make use of such checkpoints. Specifically, we'll show how to obtain the list of all of the stored checkpoints, filter this list, and even how to re-run a `Workflow` from any one them.

## Defining the Workflow

In [ ]:
import os

api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
from llama_index.core.workflow import (
    Workflow,
    step,
    StartEvent,
    StopEvent,
    Event,
    Context,
)
from llama_index.llms.openai import OpenAI


class JokeEvent(Event):
    joke: str


class JokeFlow(Workflow):
    llm = OpenAI(api_key=api_key)

    @step
    async def generate_joke(self, ev: StartEvent) -> JokeEvent:
        topic = ev.topic

        prompt = f"Write your best joke about {topic}."
        response = await self.llm.acomplete(prompt)
        return JokeEvent(joke=str(response))

    @step
    async def critique_joke(self, ev: JokeEvent) -> StopEvent:
        joke = ev.joke

        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await self.llm.acomplete(prompt)
        return StopEvent(result=str(response))

### Running Without Automatic Checkpointing

By default, automatic checkpointing is disabled. However, this can be enabled for any run via the `store_checkpoints` parameter, which we will show later in this notebook.

In [ ]:
# instantiate Jokeflow
workflow = JokeFlow()
ctx = Context(workflow=workflow)

handler = workflow.run(topic="chemistry", ctx=ctx, store_checkpoints=False)
await handler

'This joke plays on the double meaning of the word "nitrates," which can refer to both a chemical compound and a form of payment for services rendered. The humor lies in the unexpected twist of associating a scientific term with a financial concept.\n\nOne strength of this joke is its clever wordplay and the way it combines two unrelated concepts in a humorous way. The punchline is unexpected and plays on the audience\'s knowledge of chemistry and economics, making it more engaging for those who understand the reference.\n\nHowever, one potential weakness of this joke is that it may not be easily understood by those who are not familiar with chemistry or the concept of day rates. The humor relies heavily on the audience\'s background knowledge, which could limit its appeal to a more niche audience.\n\nOverall, this joke is a clever play on words that can be appreciated by those with a background in chemistry or economics. It effectively combines two unrelated concepts to create a humor

In [ ]:
len(ctx.checkpoints)

0

As we can see the number of stored checkpoints in our `Context` is 0.

In [ ]:
additional_topics = ["biology", "history"]

for topic in additional_topics:
    handler = workflow.run(
        ctx=ctx, topic=topic, store_checkpoints=True
    )  # same Context that was used before
    await handler

len(handler.ctx.checkpoints)

6

In [ ]:
# Filter by output event StopEvent
checkpoints_that_emit_stop_event = ctx.filter_checkpoints(
    output_event_type=StopEvent
)
len(checkpoints_that_emit_stop_event)
[ckpt.output_event.result[:150] for ckpt in checkpoints_that_emit_stop_event]

['This joke plays on the idea of a biologist and a mathematician being in a relationship, which is already a humorous concept due to the perceived diffe',
 'Analysis:\nThis joke plays on the double meaning of the word "pupils," which can refer to both the students in a classroom and the black part of the ey']

In [ ]:
# Filter by the name of last completed step
checkpoints_right_after_generate_joke_step = ctx.filter_checkpoints(
    last_completed_step="generate_joke"
)
len(checkpoints_right_after_generate_joke_step)
[
    ckpt.output_event.joke[:150]
    for ckpt in checkpoints_right_after_generate_joke_step
]

["Why did the biologist break up with the mathematician?\n\nBecause they couldn't find a common gene!",
 "Why did the history teacher go to jail?\n\nBecause he couldn't control his pupils!"]

### Running the Workflow

In [ ]:
# run the workflow again, but this time with checkpointing enabled
handler = workflow.run(ctx=ctx, topic="math", store_checkpoints=True)
await handler

"This joke plays on the mathematical concept of the equal sign, which is used to show that two quantities are the same. The humor comes from personifying the equal sign as a humble character who recognizes that it is not superior or inferior to anyone else.\n\nOne of the strengths of this joke is its clever wordplay and the unexpected twist on the concept of humility. By attributing human characteristics to a mathematical symbol, the joke creates a humorous and relatable scenario.\n\nHowever, one potential weakness of this joke is that it may be too niche or intellectual for some audiences. Not everyone may immediately grasp the connection between the equal sign and humility, which could limit the joke's appeal.\n\nOverall, this joke is a clever play on words that will likely resonate with those familiar with basic math concepts. Its humor lies in the unexpected personification of a mathematical symbol and the clever twist on the concept of humility."

### Listing the saved Checkpoints

In [ ]:
handler.ctx.checkpoints

[Checkpoint(id_='5a5a2bdc-d20f-400d-84d8-9d7244b5007e', last_completed_step=None, input_event=None, output_event=StartEvent(), ctx_state={'globals': {}, 'streaming_queue': '["{\\"__is_pydantic\\": true, \\"value\\": {\\"result\\": \\"This joke plays on the double meaning of the word \\\\\\"nitrates,\\\\\\" which can refer to both a chemical compound and a form of payment for services rendered. The humor lies in the unexpected twist of associating a scientific term with a financial concept.\\\\n\\\\nOne strength of this joke is its clever wordplay and the way it combines two unrelated concepts in a humorous way. The punchline is unexpected and plays on the audience\'s knowledge of chemistry and economics, making it more engaging for those who understand the reference.\\\\n\\\\nHowever, one potential weakness of this joke is that it may not be easily understood by those who are not familiar with chemistry or the concept of day rates. The humor relies heavily on the audience\'s background

In [ ]:
len(handler.ctx.checkpoints)

9

There are 3 stored checkpoints after this first run. The first of these checkpoints are created just before the emission of the `StartEvent`. This can be thought of as a "startup" step being completed which has a null input `Event` and whose output event is the `StartEvent`. The remaining two of these checkpoints are made at the completion of each of the two steps in this Workflow. 

### Filtering the Checkpoints

Before showcasing how to filter through checkpoints, let's first make things a bit more interesting by executing the workflow a few more times. Here, we'll run the workflow for two more additional topics. Since each entire run will result in 3 stored checkpoints each, we should have a total of 9 checkpoints after executing the below cell. 

### Re-Run Workflow from a specific checkpoint

In [ ]:
ckpt = checkpoints_right_after_generate_joke_step[1]
handler = workflow.run_from(checkpoint=ckpt, ctx=ctx, store_checkpoints=True)
await handler

'Analysis:\nThis joke plays on the double meaning of the word "pupils," which can refer to both the students in a classroom and the black part of the eye that dilates and contracts in response to light. The humor comes from the clever wordplay and the unexpected twist at the end of the joke.\n\nCritique:\nOverall, this joke is quite clever and has a good punchline. It effectively combines a common school scenario (a teacher struggling to control their students) with a pun on a medical term (pupils). The joke is simple and easy to understand, making it accessible to a wide audience. However, some may find the joke to be a bit predictable or corny, as pun-based humor can be hit or miss depending on personal taste. Additionally, the joke relies heavily on wordplay and may not be as funny to those who do not appreciate puns. Overall, while this joke may not be groundbreaking or particularly original, it is a lighthearted and amusing play on words that is likely to elicit a chuckle from man

Since we've executed from the checkpoint that represents the end of "generate_joke" step, there is only one additional checkpoint that gets stored i.e., the one that is stored after the completion of the next step, namely "critique_joke".

In [ ]:
len(ctx.checkpoints)

10